In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import lightgbm as lgb
import os
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

pd.options.display.max_columns = 500
#import pandas_profiling

In [2]:
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

In [3]:
random_state = 42
np.random.seed(random_state)

In [4]:

def augmentation(x=None,y=None,upsample_times=2):
    x_pos = x[y==1].copy()
    y_pos = y[y==1].copy()
    
    aug_x_pos = x_pos.copy()
    
    if upsample_times ==0:
        x_pos_temp = np.zeros(x_pos.shape)
        for i in range(x_pos.shape[1]):
            pos_values = x_pos.iloc[:,i].values
            np.random.shuffle(pos_values)
            x_pos_temp[:,i] = pos_values

        x_pos_temp = pd.DataFrame(x_pos_temp)
        x_pos_temp.columns = x_pos.columns  
        aug_x_pos = x_pos_temp    
        aug_y_pos = np.ones(y_pos.shape[0]) 
        aug_x_pos = aug_x_pos.append(x[y==0]) 
        aug_y_pos = np.append(aug_y_pos,y[y==0]) 
    else:
        for n in range(upsample_times):
            x_pos_temp = np.zeros(x_pos.shape)

            for i in range(x_pos.shape[1]):
                pos_values = x_pos.iloc[:,i].values
                np.random.shuffle(pos_values)
                x_pos_temp[:,i] = pos_values

            x_pos_temp = pd.DataFrame(x_pos_temp)
            x_pos_temp.columns = x_pos.columns  
            aug_x_pos = aug_x_pos.append(x_pos_temp)    

        aug_y_pos = np.ones(y_pos.shape[0]*(upsample_times+1)) 

        aug_x_pos = aug_x_pos.append(x[y==0]) 
        aug_y_pos = np.append(aug_y_pos,y[y==0]) 
        
    return aug_x_pos,aug_y_pos
    
    
    
    

In [8]:
# train_df[var_list] = np.exp(train_df[var_list])
# test_df[var_list] = np.exp(test_df[var_list])

In [6]:
n_folds = 5
random_seed = 26
upsample_times = 3
model = f'augment_{upsample_times}_times'

model_name = "{0}_{1}_folds".format(model, n_folds)
print("Model: {}".format(model_name))

Model: augment_3_times_5_folds


In [7]:
#exclusion = ['ID_code', 'target']+ rank_var_list
exclusion = ['ID_code', 'target'] 
feats = [c for c in train_df.columns if c not in exclusion]

In [9]:
clfs = []
folds = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_seed)
oof_preds = np.zeros((len(train_df), 1))
test_preds = np.zeros((len(test_df), 1))


X = train_df[feats]
y = train_df['target']
X_test = test_df[feats]
test_ids = test_df.ID_code.values


parameters = {
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    #'scale_pos_weight': 400,
    #'device' : 'gpu' ,
    'boosting': 'gbdt',
    'num_leaves': 31, #31
    'feature_fraction': 0.5,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'learning_rate': 0.05, #0.05
    'verbose': 30
    #'min_data_in_leaf': 200
}

feature_importance_df = pd.DataFrame()
for fold_, (trn_, val_) in enumerate(folds.split(X, y)):
    print("Current Fold: {}".format(fold_+1))
    trn_x, trn_y = X.iloc[trn_, :], y[trn_]
    val_x, val_y = X.iloc[val_, :], y[val_]
    trn_x, trn_y = augmentation(trn_x,trn_y,upsample_times)

    trn_lgb = lgb.Dataset(trn_x, trn_y)
    val_lgb = lgb.Dataset(val_x, val_y)
    clf = lgb.train(parameters,
                     train_set=trn_lgb,
                     #valid_sets=[valid_data_lgb,holdout_data_lgb],
                     valid_sets=[trn_lgb, val_lgb],
                     num_boost_round=3000,
                     early_stopping_rounds=50,
                     verbose_eval=False)
    


    val_pred = clf.predict(val_x)
    test_fold_pred = clf.predict(X_test)

    print("AUC = {}".format(roc_auc_score(val_y, val_pred)))
    oof_preds[val_, :] = val_pred.reshape((-1, 1))
    test_preds += test_fold_pred.reshape((-1, 1))
    
   # print('getting feature importance')
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = feats
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    
test_preds /= n_folds
roc_score = roc_auc_score(y, oof_preds.ravel())
print("Overall AUC = {}".format(roc_score))


print("Saving submission file")
sample = pd.read_csv('../data/sample_submission.csv')
sample.target = test_preds.astype(float)
sample.ID_code = test_ids
sample.to_csv('../submissions/{}_{}.csv'.format(model_name,str(roc_score)), index=False)

#display_importances(feature_importance_df)


Current Fold: 1


LightGBMError: GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1

In [169]:
trn_x, trn_y = X, y

In [142]:
trn_x.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,7521.343065,0.001129,1.484643e+05,162.877763,94911.485057,0.000093,167.117975,1.228654e+08,0.007299,313.249501,18.637953,24.097305,1.219193e+06,1.776242,6626.950350,2.124613e+06,3.137825e+02,7.178141e-04,7.252998e+01,2.180804e+13,3.760905e+04,1.106281e+07,13.185266,11.841378,1.764010e+06,6.818057e+05,0.005806,0.665445,138.462565,402.019261,7.345480e-01,401956.840541,0.020721,2.166265e+07,72547.732245,39281.148467,1.966785,2662.710852,1.063462e+02,48.148982,0.005307,1.594954e+03,106585.205937,170177.961357,113754.739286,8.965110e-04,3.733812e+02,6.716020e-07,9.141180e+06,2.054700e+02,408154.068101,4.116400e+12,0.499574,176.549272,0.473596,2.764029e+06,7.818241e+04,217.500250,7.525521,24750.475997,1.066844e+07,1.424513e+02,7.981290,0.806219,5853.528964,13792.772472,332.154426,5.522519e+09,150.039711,0.009086,2.494614e+09,1.761737,181.254116,7.044483e+03,5.291852e+18,9.616731e+07,0.095943,1.468952e+10,678.510531,1.984515e+05,844753.375141,1.021926e+06,3.925525,18.959403,0.010875,2.103566e+09,11186.914209,1.407203e+07,2976.192268,0.181427,4.837756e-10,880.596924,6.567017e+04,21835.666722,2.791529e+06,1.198655,7.818125e+03,1.149244e+07,8.796026,0.032936,1.304682e+04,6.033182e+05,3.350682e+11,4.221962,2.446087e+06,422.166590,13941.147528,2.821601e+07,1.342978e+06,3.807029e+10,7.631619,862.814741,50.103958,0.615636,12.478411,4.523563,12.827612,5.810633e+06,1.482027e-06,1442.163939,8.970034e+06,16760.831752,2.434399,2.171461,68.156055,163390.774466,1.042466e+06,0.480081,0.145949,5.107395e+06,3.727637e+05,1.431467,15534.210503,711.517491,176.372811,22.977144,6.194606e+12,4.353124e+13,16.909913,6.324237e+06,4149.321061,3.693798e-03,1.897845e+08,7.814333e+04,5996.312644,4313.047458,105440.279444,7.014509e-08,56.193439,6.087907e+07,1.101882e+08,4.805012e+04,8148.588491,1.783012e+07,3.573193e+04,5.245130,1.938616e+05,0.000002,4.425151e+07,102981.022226,5.165952e+06,203.019087,37.184797,1.542231e+02,793.140198,3.162853e+05,14.885685,0.497828,14736.755114,241.749001,0.008527,0.000219,1.166531e+09,23.408519,1.151562e+08,2.303765e+03,0.000040,9.126429e+05,0.029632,5.585645,0.000224,8237.070188,2.144947e+01,1.739486e+06,1.667564e+11,356.523444,138843.128104,2.738378e-09,4.289647e+07,1.796248,84.385872,52.678110,23.020842,5.424903,1.107405e+08,0.090918,2639.645746,5236.978558,3.551516e+05,0.335746
1,98775.018246,0.015807,1.044240e+06,218.984292,233795.013562,1145.160407,276.110183,1.515438e+07,23.261509,3245.744394,0.668178,3160.545758,1.231693e+06,4507.509217,229.178231,8.911785e+

In [170]:

x_pos = trn_x[trn_x==1].copy()
y_pos = trn_y[trn_y==1].copy()

aug_x_pos = x_pos.copy()

if upsample_times ==0:
    x_pos_temp = np.zeros(x_pos.shape)
    for i in range(x_pos.shape[1]):
        x_pos_temp[:,i] = np.random.shuffle(x_pos.iloc[:,i].values)

#    x_pos_temp = pd.DataFrame(x_pos_temp)
#     x_pos_temp.columns = x_pos.columns  
#     aug_x_pos = x_pos_temp    
#     aug_y_pos = np.ones(y_pos.shape[0]) 
#     aug_x_pos = aug_x_pos.append(x[y==0]) 
#     aug_y_pos = np.append(aug_y_pos,y[y==0]) 

In [182]:
a, b = augmentation(trn_x,trn_y,0)

In [183]:
a.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,6.063152e+02,0.016511,3.112697e+06,1729.175527,3.835289e+03,0.000714,402.180101,3.114081e+09,277.300013,2287.923691,3.222315,0.002865,8.099275e+05,5143.041937,10294.859789,1.755387e+06,39.440768,6.759845,6.344224e+03,1.015012e+00,6.822730e+04,7.072401e+00,311.095519,42.457348,50.164118,9.052804e+05,0.000114,0.247957,386.101389,52580.468054,1.333865e-08,907.505839,0.006979,9.117308e+05,52381.041426,65907.004192,4.687503,5762.922480,2.434715e+02,8.226001,6.130436e-09,8.016843e+03,194950.313010,121637.428340,6.684088e+02,4.192387e-02,1.033421e+05,0.040385,1.685428e+08,1.272069e+03,316127.217950,1.051077e+01,0.000031,331.192574,265.363345,3609.772392,8.709413e+07,516.822764,4.095955,13987.229308,8.857588e+05,7.130101e-05,0.448745,503.558554,2150.810442,23.259182,626.093675,1.559156e+03,152.200693,0.011669,4.321637e+09,1.459217,0.578799,2.716553e+11,5.461256e+12,3.757482e+10,0.003149,8.819845e+07,588.631293,1.508309e+06,0.067125,1.526129e+09,2.588027e-07,60681.854806,0.000007,3.053198e+06,126.545256,6.093479e+08,4627.629366,6.213349,2.620971e+03,1266.737530,6.969509e+03,27019.118851,9.834827e+02,1.794273,5.548205e+05,3.906395e+01,11.915023,1.024803,1.012624e+03,5.580728e+08,8.802092e+06,4.253746,2.579407e+04,54.811499,3856.826600,9.979809e+11,1.184818e+06,3.326877e+09,3329.908150,1253.005069,72.762448,3.482028e+01,30.180620,1.773580,34.855119,2.136521e+12,9.410494e-05,47.379991,1.503676e+21,8481.986853,0.005195,3.668563,274.485929,205911.140929,354974.061645,2.263246,0.108968,1.573660e+07,157361.344952,1.698423,24582.743696,1776.321621,0.000006,0.001523,5.711970e+11,6.942514e+11,0.588782,1.713880e+04,109.191148,1.033023e+06,8.074652e+02,2.079328e+03,1932.171927,1.017654,40166.958147,5.348493e+01,56.351001,5.681134e-10,2.283066e+08,1.302949e+02,8387.518610,1.161822e+06,4.366686e+07,0.902217,745194.099746,6.292922e-05,1.301223e+07,3.064209e+06,5.623925e+02,334.889290,409.075608,2.073721e+03,0.000025,8.636817e+09,13.008461,7.903455,370.332037,789.026573,19.474385,0.103343,1.255110e+15,110.686292,1.778746e+14,34420.238539,3.160976e-08,4.719768e+04,4029.912732,24.065999,26.912746,65107.823654,6.494788e+00,1.022331e-01,640561.234523,8.836185e-03,980.046481,1.538196e-06,2.430068e+08,4.174940,16.480865,82.335305,0.585201,870.441018,1.130358e+09,0.463615,18337.437241,5599.877492,3.148699e+06,5.998775e-03
1,3.325115e+07,0.019405,3.149644e+06,536.410647,2.399018e+04,0.000016,322.853933,2.908404e+07,12.200781,7057.174232,6.105561,0.823740,1.582474e+06,17061.845633,3220.848136,2.1

In [152]:
x_pos.headx

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
13,8.2230,-6.5600,7.2472,4.8399,9.0727,-16.1873,4.1387,18.0670,6.8005,7.7059,-0.4793,-11.7092,14.1270,15.1742,8.4112,14.5302,7.2952,-7.3676,25.2003,10.4003,18.0884,2.2045,11.9862,2.4217,12.6962,14.2951,-4.8168,-1.2545,5.0663,7.0250,3.2677,7.5828,2.0063,5.7809,11.5834,4.2839,-3.5209,4.9861,8.2715,4.2398,3.8990,2.7066,10.0846,11.4270,3.8853,1.4783,9.6528,-2.5373,15.3853,21.6893,12.8586,20.1357,-3.3310,6.4409,1.0700,4.5864,21.4236,6.4198,-1.2175,8.7170,15.6754,1.5946,-1.3398,-2.3787,8.3178,4.7841,5.8252,9.0716,5.0220,1.3883,41.2607,0.7271,4.4890,22.2617,17.5342,7.7751,11.7045,18.6023,7.1760,18.0417,11.5840,14.6044,-0.3173,15.0602,-9.5512,12.2299,14.1932,3.5846,7.4539,5.0796,-32.9253,6.9113,11.6188,10.8263,13.0835,1.0063,10.5730,20.1437,0.8847,-1.0340,-1.7285,13.3732,21.8984,1.6978,13.1572,5.3445,8.8985,9.3201,14.5399,12.6124,5.8613,4.7920,3.6867,4.4490,3.3867,5.5250,3.8392,27.9318,9.1243,0.8445,9.6006,10.1863,-4.3799,16.8190,1.1766,13.2250,12.8256,3.8281,-0.9617,13.7343,10.7141,1.1014,6.3886,6.3341,1.1738,-11.6993,6.0963,6.2875,4.6631,13.3914,-0.7507,8.0948,12.8023,11.8899,8.8824,6.8846,7.5306,-9.7267,3.5775,-0.6343,14.5192,19.5337,12.8833,20.6827,5.8083,-6.4582,12.6233,1.2555,26.3368,4.3244,41.5631,5.5767,6.5073,4.9681,0.6612,18.9046,2.5230,-10.0047,2.2251,5.3125,5.2284,3.4643,-1.1958,2.5592,32.8121,10.1209,8.5747,6.8069,26.3741,3.0913,-5.0768,9.8165,-2.9793,2.4957,19.8129,-2.4520,7.3811,-13.2933,17.9164,0.7868,4.4272,10.7419,-0.3146,-6.2010,16.0088,-1.4198,-7.9110,9.1139,20.8679,0.8269
63,12.2656,3.0087,14.7130,7.7768,8.3993,-19.1143,5.0314,16.8659,2.4612,9.0093,6.1630,-3.1952,14.0663,7.2131,10.1751,13.9092,7.6918,-1.3902,25.4251,9.2694,5.0573,31.9582,2.2689,2.8250,10.8028,13.9218,-7.3145,-3.4382,3.8495,2.8307,-15.4858,14.1487,2.1338,15.0303,10.6422,-1.7747,7.5306,4.4585,13.8119,4.0529,-17.4062,14.0290,11.3079,11.5014,-0.4756,-30.2509,18.1156,-2.4032,3.6718,17.8801,14.1845,19.9383,-1.1048,5.8205,-11.0088,16.6013,6.4307,6.2821,1.2881,9.5146,13.6054,-12.7409,0.0269,-2.4077,6.7636,3.3215,5.9005,18.7263,5.0321,-1.8527,15.3190,0.3458,-3.3172,13.3337,28.5337,12.9415,4.7370,18.7461,7.0523,11.9636,2.7108,16.9357,-10.1209,-10.2051,-11.0086,12.3040,-2.6024,12.7008,6.6054,3.2648,-39.3756,6.9868,15.7135,10.4254,8.9022,-0.6451,12.1135,30.2472,1.5942,3.9082,1.1945,12.6851,3.5259,1.7921,13.0434,6.1506,9.0080,9.0438,14.2872,24.9922,6.0128,7.7370,3.3981,0.7656,4.2242,4.5912,1.4718,7.8898,-17.7512,-0.0470,40.7348,12.2305,-7.1032,14.3320,7.1812,12.7844,12.1478,1.8752,1.4461,14.2797,13

In [197]:
%%javascript
var nb = IPython.notebook;
var kernel = IPython.notebook.kernel;
var command = "NOTEBOOK_FULL_PATH = '" + nb.base_url + nb.notebook_path + "'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [198]:


shutil.copyfile(os.path.basename(NOTEBOOK_FULL_PATH), 
                             '../models/{}_{}.ipynb'.format(model_name, str(roc_score)))


'../models/augment_3_times_5_folds_0.8965966827580265.ipynb'